In [12]:
import torch_geometric as pyg
import torch

import model

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
list(x.name for x in gat.parameters())

[None, None, None]

In [4]:
list(gat.named_buffers())

[]

In [3]:
N = 10
E = 20
in_size = 4
channels = 16

In [4]:
x = torch.randn(N, in_size)
edge_index = torch.randint(0, N, (2, E))

In [8]:
attn = torch.nn.Linear(2 * in_size, channels)

In [9]:
out = attn(torch.cat([x[edge_index[0]], x[edge_index[1]]], dim=-1))

In [10]:
out.shape

torch.Size([20, 16])

In [11]:
out.shape

torch.Size([20, 16])

In [12]:
a = torch.randn(20)

In [13]:
a.shape

torch.Size([20])

In [14]:
(a @ out).shape

torch.Size([16])

In [21]:
layer = model.GATLayer(in_size, channels, heads=2, concat=True)
gat = pyg.nn.GATConv(in_size, channels, heads=2, bias=False, add_self_loops=False)

In [22]:
[(n, p.shape) for n, p in layer.named_parameters()]

[('lin.weight', torch.Size([32, 4])), ('attn.0.weight', torch.Size([2, 32]))]

In [23]:
[(n, p.shape) for n, p in gat.named_parameters()]

[('att_src', torch.Size([1, 2, 16])),
 ('att_dst', torch.Size([1, 2, 16])),
 ('lin.weight', torch.Size([32, 4]))]

In [16]:
# do frankenstein to verify the implementations are equivalent
with torch.no_grad():
    gat.lin.weight.copy_(layer.lin.weight)
    # map attn_src, attn_dst to layer.attn.weight[:16], layer.attn.weight[16:]
    gat.att_src.copy_(layer.attn[0].weight[:, :16].unsqueeze(0))
    gat.att_dst.copy_(layer.attn[0].weight[:, 16:].unsqueeze(0))

RuntimeError: The size of tensor a (32) must match the size of tensor b (16) at non-singleton dimension 0

In [24]:
with torch.no_grad():
    out1 = layer(x, edge_index)
    out2 = gat(x, edge_index)

RuntimeError: shape '[30, 2, 1]' is invalid for input of size 120

In [49]:
torch.allclose(out1, out2)

False

In [41]:
out1.shape, out2.shape

(torch.Size([10, 16]), torch.Size([10, 16]))

In [48]:
out1[0], out2[0]

(tensor([-0.0075,  0.0428, -0.0291, -0.0515,  0.0149,  0.0191, -0.0222, -0.0049,
         -0.0160,  0.0271,  0.0102, -0.0013,  0.0140, -0.0598, -0.0263,  0.0311]),
 tensor([-0.1495,  0.8535, -0.5807, -1.0273,  0.2971,  0.3807, -0.4422, -0.0987,
         -0.3187,  0.5410,  0.2038, -0.0269,  0.2789, -1.1940, -0.5244,  0.6200]))

In [43]:
torch.allclose(gat.lin.weight, layer.lin.weight)

True

In [44]:
torch.allclose(gat.att_src, layer.attn[0].weight[:, :16].unsqueeze(0))

True

In [45]:
torch.allclose(gat.att_dst, layer.attn[0].weight[:, 16:].unsqueeze(0))

True

In [46]:
out1[:, 0], out2[:, 0]

(tensor([-0.0098, -0.0062, -0.0109, -0.0161, -0.0417,  0.0073, -0.0818,  0.0187,
          0.0365,  0.0867]),
 tensor([-0.1495,  0.0000, -0.5207,  0.0000, -0.1948, -0.1339,  0.1864,  0.0885,
          0.0000,  0.3499]))

In [25]:
with torch.no_grad():
    out = layer(x, edge_index)

RuntimeError: shape '[30, 2, 1]' is invalid for input of size 120

In [11]:
out.shape

torch.Size([10, 16])